In [1]:
import numpy as np
import pandas as pd

from os.path import join
from os import makedirs
from glob import glob
import re

from tqdm import tqdm

import scipy.io

import matplotlib.pyplot as plt
import seaborn as sns

---

In [2]:
def convert_ID(ID):
    ##################   ##################
    #  1  2  3  4  5 #   #        2       #
    #  6  7  8  9 10 #   #        1       #
    # 11 12 13 14 15 # = # -2 -1  0  1  2 #
    # 16 17 18 19 20 #   #       -1       #
    # 21 22 23 24 25 #   #       -2       #
    ##################   ##################
    x = np.kron(np.ones(5),np.arange(-2,3)).astype(int)
    y = np.kron(np.arange(2,-3,-1),np.ones(5)).astype(int)
    pos = np.array((x[ID-1],y[ID-1]))
    return pos

In [3]:
sec=4
ntrial = 12
nblock = 12
nrun=6

In [4]:
def func_AMregressor(datum):
    nS = int(datum['nSampleTrial'][0][0]) # 4 s * 60 Hz = 240 samples
    assert sec*60==nS

    tpr = datum['nTrialperRun'][0][0]     # the number of trials per run = 1 trial+ 12 trial/block * 12 blocks
    assert 1+ntrial*nblock==tpr

    onsettime = datum['LearnTrialStartTime'][0]
    idx_editpoint = [i+1 for i,t in enumerate(onsettime[:-2]) if (onsettime[i]>onsettime[i+1])]
    assert (np.diff(idx_editpoint)==tpr).all() # Are you sure the fact that 145 trials per run?

    nrun = int(datum['nRun'][0][0])       # the number of runs = 6 runs
    targetID = datum['targetID'][0][idx_editpoint[0]:] # targetID.shape = 870 trials = 145 trial/run * 6 runs
    assert targetID.shape==tpr*nrun

    ## onset times
    tmp = np.zeros((nrun, tpr), dtype=float)
    for run in range(nrun):
        idx = idx_editpoint[run]
        tmp[run,:] = onsettime[idx:idx+tpr]*0.001
    onsettime=tmp

    ## counting how many times did they hit the target
    hit_or_not = np.zeros((tpr*nrun, nS), dtype=bool) # hit_or_not.shape = (# of trials/run, # if frames/trial)
    ## The total distance the cursor moved during one trial
    cursor_length = np.zeros(tpr*nrun, dtype=float)
    for t, ID in enumerate(targetID):
        pos = datum['boxSize']*convert_ID(ID) # r_target = [x_target, y_target]
        ## allXY.shape = (2, 60 Hz * 4 s/trial * 145 trials/run * 6 runs = 208800 frames)
        xy = datum['allXY'][:,nS*t:nS*(t+1)] # r_cursor = [x_cursor, y_cursor]
        dr = np.diff(xy, axis=1)
        assert dr.shape==(2,nS-1)
        cursor_length[t] = np.matmul(dr.T, dr).sum()
        ## err.shape = (2, nS)
        err = xy - np.ones((2,nS))*pos.T # dr = r_cursor - r_target
        ## is the cursor in the target box?
        hit_or_not[t,:] = (abs(err[0,:]) <= datum['boxSize']*0.5) & (abs(err[1,:]) <= datum['boxSize']*0.5)

    cnt_hit = hit_or_not.reshape(nrun, tpr, sec, 60).sum(axis=(2,3))
    cursor_length = cursor_length.reshape((nrun,tpr))

    return onsettime, cnt_hit, cursor_length

---

In [5]:
list_nn = [
    '03','04','05','06','07',
    '08','09','10','11','12',
    '14','15','16','17','18',
    '19','20','21','22','24',
    '25','26','27','29'
]

---

In [6]:
# dir_root = '/mnt/ext4/GL'
dir_root = '/home/sungbeenpark/Github/labs/GL'

dir_behav = join(dir_root,'behav_data')
dir_reg = join(dir_behav,'regressors')
dir_AM = join(dir_reg,'AM')
makedirs(dir_AM, exist_ok=True)

---

In [7]:
for nn in tqdm(list_nn):
    subj = 'GL%s'%nn
    datum = scipy.io.loadmat(join(dir_behav, '%s-fmri.mat'%subj))
    
    onsettime, cnt_hit, cursor_length = func_AMregressor(datum)
    
    ## Extract the start time of the trial regardless of the on/off condition
    np.savetxt(
        join(dir_reg, '%s.onsettime.txt'%subj),
        X=onsettime, fmt='%.3f', delimiter=' ', newline='\n'
    )
    
    ## Exclude the first trial and divide into blocks and trials in a main task.
    run=4
    tmp = onsettime[:run,1:].reshape(run,nblock,ntrial)
    
    ## Print onset time only for the trials with the 'On' condition in the main task.
    np.savetxt(
        join(dir_reg, '%s.onsettime.main_On.txt'%subj),
        X=tmp[:,::2,:].reshape(run,-1),
        fmt='%.3f', delimiter=' ', newline='\n'
    )
    
    ## Print onset time only for the trials with the 'Off' condition in the main task.
    np.savetxt(
        join(dir_reg, '%s.onsettime.main_Off.txt'%subj),
        X=tmp[:,1::2,:].reshape(run,-1),
        fmt='%.3f', delimiter=' ', newline='\n'
    )
    
    ## Print onset time of the test session
    tmp = onsettime[run:,:]
    np.savetxt(
        join(dir_reg, '%s.onsettime.test.txt'%subj),
        X=tmp,
        fmt='%.3f', delimiter=' ', newline='\n'
    )
    
    ## Reward
    reward = cnt_hit/(sec*60)
    
    for shift in [0, 2, 4]:
        ## AM regressor
        AM = [[] for ii in range(nrun)]
        for run in range(nrun):
            AM[run] = ['%.3f*%.3f'%(o,r) for o,r in zip(onsettime[run][1:]+shift, reward[run][1:])]
        AM = np.array(AM)

        ## AM regressor for the 'On' condition
        np.savetxt(
            join(dir_AM, '%s.reward.main_On.shift=%.1f.txt'%(subj,shift)),
            X=AM[:4].reshape(4,nblock,ntrial)[:,::2,:].reshape(4,-1),
            fmt='%s', delimiter=' ', newline='\n'
        )

        ## AM regressor for the 'Off' condition
        np.savetxt(
            join(dir_AM, '%s.reward.main_Off.shift=%.1f.txt'%(subj,shift)),
            X=AM[:4].reshape(4,nblock,ntrial)[:,1::2,:].reshape(4,-1),
            fmt='%s', delimiter=' ', newline='\n'
        )

        ## AM regressor for the test session
        np.savetxt(
            join(dir_AM, '%s.reward.test.shift=%.1f.txt'%(subj,shift)),
            X=AM[4:,:],
            fmt='%s', delimiter=' ', newline='\n'
        )
        
    ## Cursor Movement (scaled)
#     mean = cursor_length.mean(axis=1)
    mean = 0
    std = cursor_length.std(axis=1)
    for rr in range(nrun):
        cursor_length[rr] = (cursor_length[rr]-mean[rr])/std[rr]

    shift = 0
    ## AM regressor
    AM = [[] for ii in range(nrun)]
    for run in range(nrun):
        AM[run] = ['%.3f*%.3f'%(o,r) for o,r in zip(onsettime[run][1:]+shift, cursor_length[run][1:])]
    AM = np.array(AM)
    
    np.savetxt(
        join(dir_AM, '%s.cursor_length.txt'%(subj)),
        X=AM[:4],
        fmt='%s', delimiter=' ', newline='\n'
    )

100%|██████████| 24/24 [00:03<00:00,  6.36it/s]
